In [1]:
import json
import os
import pandas as pd
import re
from src.data.process_data import process_open_ended, process_wave_data
from src.data.read_data import load_raw_survey_data, read_stata_file
from src.paths import CODING_DIR, GLES_DIR, PROCESSED_DATA_DIR, PROJECT_DIR, PROMPT_DIR, RAW_DATA_DIR
from src.utils import format_prompt


In [2]:

wave_number = 12
prompt_fpath=os.path.join(PROMPT_DIR,'10_prompt_with_vocational_training.txt')  
wave_df, wave_open_ended_df, df_coding_840s = load_raw_survey_data(wave_number)
wave_open_ended_df = process_open_ended(wave_open_ended_df, df_coding_840s, wave_number)
wave_df_processed = process_wave_data(wave_df, wave_open_ended_df, wave_number)
wave_df_processed['formatted_prompt'] = wave_df_processed.apply(lambda row: format_prompt(prompt_fpath,row), axis=1)

In [12]:
(wave_df.shape[0]-.shape[0] )/ wave_df_processed.shape[0]

0.052439286481839674

In [ ]:
import pandas as pd

a= pd.read_csv('/dss/dsshome1/0F/ra46lup2/ma-yoztyurk/ZA7957_7707_v2.0.0/ZA7957_6838_v2.0.0.csv',sep=';').filter(regex=r'kp12_840_c').ge(0).astype(int).sum(axis=1)
a= a[a>0]
a.value_counts(1)

In [ ]:
for i in range(10,21):
    a= pd.read_csv('/dss/dsshome1/0F/ra46lup2/ma-yoztyurk/ZA7957_7707_v2.0.0/ZA7957_6838_v2.0.0.csv',sep=';').filter(regex=f'kp{i}_840_c').ge(0).astype(int).sum(axis=1)
    a= a[a>0]
    print('wave ',i)
    print('%',a.value_counts(1).iloc[0] *100 ,'of answers have 1 label in wave',i)
    print('%',a.value_counts(1).iloc[1] *100 ,'of answers have 2 labels in wave',i)
    print('%',a.value_counts(1).iloc[2] *100 ,'of answers have 3 labels in wave',i)

    print('- - - - - - -')


In [ ]:
#pick last 20 cols
wave_df_processed= wave_df_processed[wave_df_processed.kp12_2090a.notna()]
wave_df_processed= wave_df_processed[wave_df_processed.kpx_840_class1_name.notna()]

wave_df_processed[wave_df_processed['kpx_840_text'].str.len()>20].filter(regex='lfdn$|^age|gender|leaning_party|ostwest|edu|clause|kpx_840_text|kpx_840_class1_name')#.kp12_2090a.value_counts()
#filter where len of kpx_840_text > 20 and kpx_840_text is not null


In [ ]:
from pprint import pprint

ll=[]
for row in wave_df_processed.formatted_prompt.iloc[40:55].values:
    ll.append( row.split("<</SYS>>",1)[1].strip(r'[/INST]')[:-9])
    #print(row)
    print( row.split("<</SYS>>",1)[1].strip(r'[/INST]')[:-9])

In [ ]:
 

cols_to_stratify = [
'ostwest',
 'leaning_party',
 'gender',
 'age_group',
]

strata_counts = wave_df_processed.groupby(cols_to_stratify).size().reset_index(name='counts').sort_values(by="counts")
strata_counts